In [ ]:
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from matplotlib import pyplot
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
import re

from sklearn.preprocessing import LabelEncoder

Using TensorFlow backend.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
cd drive/My\ Drive/Research/NLP

/content/drive/My Drive/Research/NLP


In [ ]:
print(data[data['sentiment'] == 'Positive'].size)
print(data[data['sentiment'] == 'Negative'].size)

4472
16986


In [ ]:
data = pd.read_csv('Sentiment.csv')
# Keeping only the neccessary columns
data = data[['text','sentiment']]

data['text'] = data['text'].apply(lambda x: x.lower())
data['text'] = data['text'].apply((lambda x: re.sub('[^a-zA-z0-9\s]', '', x)))

for idx, row in data.iterrows():
    row[0] = row[0].replace('rt', ' ')

max_fatures = 2000
tokenizer = Tokenizer(num_words=max_fatures, split=' ')
tokenizer.fit_on_texts(data['text'].values)
X = tokenizer.texts_to_sequences(data['text'].values)

X = pad_sequences(X)

embed_dim = 128
lstm_out = 196
def createmodel():
    model = Sequential()
    model.add(Embedding(max_fatures, embed_dim,input_length = X.shape[1]))
    model.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2))
    model.add(Dense(3,activation='softmax'))
    model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
    return model
# print(model.summary())

labelencoder = LabelEncoder()
integer_encoded = labelencoder.fit_transform(data['sentiment'])
y = to_categorical(integer_encoded)
X_train, X_test, Y_train, Y_test = train_test_split(X,y, test_size = 0.33, random_state = 42)

batch_size = 32
model = createmodel()
model.fit(X_train, Y_train, epochs = 1, batch_size=batch_size, verbose = 2)
score,acc = model.evaluate(X_test,Y_test,verbose=2,batch_size=batch_size)
print(score)
print(acc)
print(model.metrics_names)

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/1
 - 30s - loss: 0.8348 - accuracy: 0.6440
0.7528762072780446
0.6743119359016418
['loss', 'accuracy']


In [ ]:
model.save('sentiment_model.h5')

In [ ]:
from keras.models import load_model
model = load_model('sentiment_model.h5')

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


In [ ]:
def predict(input_sentence):
    tokenizer.fit_on_texts(input_sentence)
    X = tokenizer.texts_to_sequences(input_sentence)
    X = pad_sequences(X, maxlen=28)
    sentiment = model.predict(X, batch_size=1, verbose=2)[0]
    print(sentiment)

In [ ]:
predict("dull dumb waste bad thing")

[0.23464464 0.5809486  0.18440676]


In [ ]:
predict("good awesome great super excited crazy really great idea")

[0.24654879 0.5795063  0.17394494]


In [ ]:
predict("I exist")

[0.17690544 0.5656382  0.2574564 ]


Model 2

In [ ]:
def create_new_model():
    model = Sequential()
    model.add(Embedding(max_fatures, embed_dim, input_length=X.shape[1]))
    model.add(SpatialDropout1D(0.4))
    model.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2))
    model.add(Dense(2, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    model.save('lstm.h5')
    return model

In [ ]:
model2 = create_new_model()

In [ ]:
data = data[['text', 'sentiment']]
data = data[data.sentiment != 'Neutral']

data = data[data.sentiment != "Neutral"]
data['text'] = data['text'].apply(lambda x: x.lower())
data['text'] = data['text'].apply((lambda x: re.sub('[^a-zA-z0-9\s]', '', x)))

print(data[data['sentiment'] == 'Positive'].size)
print(data[data['sentiment'] == 'Negative'].size)

for idx, row in data.iterrows():
    row[0] = row[0].replace('rt', ' ')

max_fatures = 2000
tokenizer = Tokenizer(num_words=max_fatures, split=' ')
tokenizer.fit_on_texts(data['text'].values)
X = tokenizer.texts_to_sequences(data['text'].values)
print(X)
X = pad_sequences(X)
print(X)
embed_dim = 128
lstm_out = 196


def createmodel():
    model = Sequential()
    model.add(Embedding(max_fatures, embed_dim, input_length=X.shape[1]))
    model.add(SpatialDropout1D(0.4))
    model.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2))
    model.add(Dense(2, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    model.save('lstm.h5')
    return model


# print(model.summary())

labelencoder = LabelEncoder()
integer_encoded = labelencoder.fit_transform(data['sentiment'])
y = to_categorical(integer_encoded)
X_train, X_test, Y_train, Y_test = train_test_split(X, y, test_size=0.33, random_state=42)
print(X_train.shape, Y_train.shape)
print(X_test.shape, Y_test.shape)

batch_size = 32
model = createmodel()
model.fit(X_train, Y_train, epochs=5, batch_size=batch_size, verbose=2)
score, acc = model.evaluate(X_test, Y_test, verbose=2, batch_size=batch_size)
print(score)
print(acc)

4472
16986
[[363, 122, 1, 703, 2, 39, 58, 237, 37, 210, 6, 174, 1761, 12, 1324, 1409, 743], [16, 284, 252, 5, 821, 102, 167, 26, 136, 6, 1, 173, 12, 2, 233, 724, 17], [1261, 2, 303, 23, 1943, 1, 1632, 216, 12, 1, 704, 6, 185, 207, 371, 670], [127, 17, 53, 263, 410, 9, 82, 304, 441, 1325, 1762, 1150, 62, 1944, 194, 2, 51], [9, 1151, 167, 8, 21, 1326, 63, 9, 614, 188, 21, 189, 4, 34, 1, 563, 19, 822, 2, 44, 744], [12, 1, 168, 96, 547, 34, 1, 103, 589, 52, 59, 1763, 7, 17, 615, 2], [39, 149, 26, 823, 13, 2, 926], [23, 12, 46, 16], [36, 1945, 47, 4, 860, 20, 69, 2, 174, 548, 6, 1, 58, 275, 335], [31, 1208, 53, 453, 22, 54, 146, 1, 2, 21], [671, 9, 528, 305, 169, 55, 12, 1327, 182, 13, 47, 2], [306, 372, 1, 6, 893, 14, 92, 466, 270, 5, 271, 2], [266, 467, 966, 14, 453, 22, 107, 12, 17, 2, 475], [616, 52, 186, 132, 175, 109, 29, 1, 2, 51, 277, 45, 6, 109], [770, 894, 5, 582, 12, 1328, 140, 1012, 5, 1, 42, 154, 37, 1, 770, 1, 603, 434, 795, 14, 1, 49, 2], [1410, 118, 1329, 14, 1633, 14, 1764,

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/5
 - 22s - loss: 0.4415 - accuracy: 0.8134
Epoch 2/5
 - 22s - loss: 0.3230 - accuracy: 0.8639
Epoch 3/5
 - 22s - loss: 0.2826 - accuracy: 0.8812
Epoch 4/5
 - 22s - loss: 0.2536 - accuracy: 0.8979
Epoch 5/5
 - 22s - loss: 0.2242 - accuracy: 0.9108
0.35705159130287656
0.8523015975952148


In [ ]:
model = load_model('lstm.h5')


def predict(input_sentence):
    # sentence = "A lot of good things are happening. We are respected again throughout the world, and that's a great thing"
    max_fatures = 2000
    tokenizer = Tokenizer(num_words=max_fatures, split=' ')
    tokenizer.fit_on_texts(input_sentence)
    X = tokenizer.texts_to_sequences(input_sentence)
    X = pad_sequences(X, maxlen=28)
    sentiment = model.predict(X, batch_size=1, verbose=2)[0]
    print(sentiment)



predict("A lot of good things are happening. We are respected again throughout the world, and that's a great thing")

[0.50963235 0.4903676 ]


## Task 2
GridSearch CV

In [ ]:
import numpy as np  # linear algebra
from keras.callbacks import TensorBoard
from keras.wrappers.scikit_learn import KerasClassifier

model = KerasClassifier(build_fn=createmodel, verbose=0)
batch_size = [32]
epochs = [1, 2]
param_grid = dict(batch_size=batch_size, epochs=epochs)

In [ ]:
from sklearn.model_selection import GridSearchCV

grid = GridSearchCV(estimator=model, param_grid=param_grid)

In [39]:
tensorboard = TensorBoard(log_dir="/content/sample_data",histogram_freq=0, write_graph=True, write_images=True)
# model.fit(X_train, Y_train, nb_epoch=epochs, batch_size=32,callbacks=[tensorboard])

In [33]:
grid_result = grid.fit(X_train[:1000], Y_train[:1000])

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


In [35]:
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

Best: 0.800000 using {'batch_size': 32, 'epochs': 1}


### Task 3
Apply the code on spamdata set

In [ ]:
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
data = pd.read_csv('spam.csv', encoding='ISO-8859-1')
# Keeping only the neccessary columns
data = data[['v1', 'v2']]

data['v2'] = data['v2'].apply(lambda x: re.sub('[^a-zA-z0-9\s]', '', x.lower()))

max_fatures = 2000
tokenizer = Tokenizer(num_words=max_fatures, split=' ')
tokenizer.fit_on_texts(data['v2'].values)
X = tokenizer.texts_to_sequences(data['v2'].values)
print(X)
X = pad_sequences(X)
print(X)
embed_dim = 128
lstm_out = 196


def createmodel():
    model = Sequential()
    model.add(Embedding(max_fatures, embed_dim, input_length=X.shape[1]))
    model.add(SpatialDropout1D(0.4))
    model.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2))
    model.add(Dense(2, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model


labelencoder = LabelEncoder()
integer_encoded = labelencoder.fit_transform(data['v1'])

y = to_categorical(integer_encoded)
X_train, X_test, Y_train, Y_test = train_test_split(X, y, test_size=0.33, random_state=42)
print(X_train.shape, Y_train.shape)
print(X_test.shape, Y_test.shape)

batch_size = 32
model = createmodel()
model.fit(X_train, Y_train, epochs=5, batch_size=batch_size, verbose=2)
score, acc = model.evaluate(X_test, Y_test, verbose=2, batch_size=batch_size)
print(score)
print(acc)

[[46, 442, 794, 711, 680, 64, 9, 1245, 90, 119, 356, 1246, 154, 1247, 67, 58, 137], [48, 312, 1399, 443, 6, 1823], [50, 459, 9, 22, 4, 749, 899, 1, 179, 1824, 1131, 625, 1825, 261, 71, 1824, 1, 1826, 1, 313, 459, 79, 382], [6, 229, 142, 24, 357, 6, 160, 143, 60, 142], [949, 2, 43, 97, 72, 460, 1, 900, 72, 1827, 202, 111, 461], [844, 120, 67, 1588, 41, 100, 184, 597, 21, 7, 39, 343, 86, 358, 56, 109, 383, 3, 42, 12, 15, 84, 1828, 48, 369, 1034, 1, 70, 302, 1], [203, 11, 598, 8, 25, 56, 1, 359, 35, 10, 108, 681, 10, 56], [73, 216, 13, 1132, 1400, 1400, 1829, 112, 100, 573, 73, 13, 950, 12, 51, 1589, 951, 462, 1, 1035, 13, 217, 950], [682, 73, 4, 795, 413, 218, 3, 17, 100, 414, 1, 1036, 140, 952, 1, 115, 16, 115, 401, 504, 712, 548, 64], [129, 13, 92, 1037, 796, 27, 124, 6, 85, 1133, 1, 463, 1, 5, 303, 525, 845, 35, 329, 12, 50, 16, 5, 92, 463, 1248, 50, 18], [23, 219, 32, 80, 209, 7, 2, 43, 69, 1, 264, 77, 40, 276, 1134, 210, 180, 168, 415, 91], [1830, 1, 179, 173, 47, 314, 1, 599, 79, 7

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/5
 - 60s - loss: 0.2093 - accuracy: 0.9309
Epoch 2/5
 - 60s - loss: 0.0522 - accuracy: 0.9842
Epoch 3/5


In [41]:
def predict(input_sentence):
    # sentence = "A lot of good things are happening. We are respected again throughout the world, and that's a great thing"
    max_fatures = 2000
    tokenizer = Tokenizer(num_words=max_fatures, split=' ')
    tokenizer.fit_on_texts(input_sentence)
    X = tokenizer.texts_to_sequences(input_sentence)
    X = pad_sequences(X, maxlen=152)
    sentiment = model.predict(X, batch_size=1, verbose=2)[0]
    print(sentiment)

predict("click here to become a millionaire overnight")

[0.988308   0.01169204]
